In [1]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 90)
%config Completer.use_jedi=False

# ChEMBL API client

In [117]:
assay = new_client.assay

In [118]:
ids = pd.read_csv('chembl_ids.csv')
ids=ids.sort_values('ChEMBL').reset_index(drop=True).astype(str)
ids.head(2)

,ChEMBL
0,654926
1,688213


In [140]:
kinases=pd.read_csv('pipeline_test_kinase_all.csv', index_col=0)

In [141]:
targets=[kinase.replace('CHEMBL','') for kinase in kinases.columns]
targets.pop(0)
targets.pop(0)
targets.sort()

targs = pd.DataFrame(targets)
targs.columns=['PQSAR']
targets=['CHEMBL'+ targ for targ in targets]

print(ids.shape, targs.shape)
ids.ChEMBL.astype(str).equals(targs.PQSAR.astype(str))

(159, 1) (159, 1)


True

In [142]:
cols=['Compound ID', 'Structure']
cols.extend(targets)
kinases=kinases[cols]
kinases.head()

,Compound ID,Structure,CHEMBL654926,CHEMBL688213,CHEMBL688293,CHEMBL688395,CHEMBL688427,CHEMBL688624,CHEMBL688660,CHEMBL688781,...,CHEMBL809276,CHEMBL809277,CHEMBL809278,CHEMBL809279,CHEMBL809280,CHEMBL809281,CHEMBL809282,CHEMBL809283,CHEMBL809284,CHEMBL809285
0,PWEDOA,CC(C)=C(C)c1cccc2c(CCCOc3cccc(Cl)c3Cl)c(C(=O)O...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.5,4.2,NaN,4.3,4.4,5.7,4.1,4.2,4.0
1,PWEDAD,CC1=CC(=CC=C1)N=C(O)NC2=CC=C(C=C2)C3=CSC4=C3[C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,4.1,5.7,NaN,7.5,4.4,6.1,4.1,4.2,4.0
2,PWEDGC,CC1=N[NH]C2=CC=CC(=C12)C3=CC=C(NC(O)=NC4=CC(=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.7,5.8,4.2,NaN,6.7,4.4,5.2,4.1,4.2,6.0
3,PWEDOQ,CC1=CC(=C(C)C(=C1)C)OCCCC2=C([NH]C3=C2C=CC=C3C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.2,NaN,4.3,4.4,5.6,4.1,4.2,4.0
4,PWEDCP,C1CC(CCO1)C[N]2C=C(N=N2)C3=CC4=C[NH]N=C4C=C3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.7,4.1,6.1,NaN,4.3,4.4,4.1,4.1,4.2,7.5


In [153]:
target_ids={}
for targ in targets:
    # print(targ)
    target_chembl_id=assay.filter(assay_chembl_id=targ).only('target_chembl_id')[0]['target_chembl_id']
    target_info = target.filter(target_chembl_id=target_chembl_id).only(['pref_name', 'target_components'])[0]
    target_name = target_info['pref_name']
    target_syn=target_info['target_components'][0]['target_component_synonyms']
    target_syn = [d for d in target_syn if d['syn_type'] in keyValList]
    target_syn = target_syn[0]['component_synonym']
    targdict={'name':target_name, 'symbol':target_syn}
    target_ids[targ]=targdict
    # print('\t fin')

IndexError: list index out of range

In [171]:
targ=targets[0]

In [172]:
len(kinases[~kinases[targ].isna()])

207

In [173]:
activity = new_client.activity
herg_activities = activity.filter(assay_chembl_id=targ)
len(herg_activities)

2

In [158]:
target_info = target.filter(target_chembl_id=target_chembl_id)

In [159]:
target_info

[{'cross_references': [], 'organism': 'Cavia porcellus', 'pref_name': 'Cavia porcellus', 'species_group_flag': False, 'target_chembl_id': 'CHEMBL369', 'target_components': [], 'target_type': 'ORGANISM', 'tax_id': 10141}]

In [160]:
targ

'CHEMBL688213'

In [177]:
test = target_ids['NN']

KeyError: 'NN'

# ChEMBL data reorg

In [324]:
st_type='Ki'
with pd.read_csv("ChEMBL_Kinases.csv", sep=';', chunksize=10000) as reader:
    for i, chunk in enumerate(reader):
        chunk.columns=[col.replace(' ','_') for col in chunk.columns]
        chunk=chunk[~chunk.pChEMBL_Value.isna()]
        chunk=chunk[chunk.Standard_Type==st_type]
        chunk=chunk[['Molecule_ChEMBL_ID', 'Smiles', 'Standard_Value','Target_ChEMBL_ID', 'Target_Name']]
        chunk['Kinase']=np.nan
        chunk.loc[chunk.Kinase.isna(), 'Kinase'] = chunk.Target_ChEMBL_ID+ '_' + chunk.Target_Name
        chunk=chunk.drop(columns=['Target_ChEMBL_ID', 'Target_Name'])
        chunk=chunk.pivot_table(index=['Molecule_ChEMBL_ID', 'Smiles',], columns=['Kinase'], values='Standard_Value')
        # chunk=chunk.reset_index(1)
        print(chunk.shape)
        chunk.to_csv(f'./chunks/chembl_kinases_pivot_{st_type}_nM_{i}.csv')
        if i==0:
            fulldf = chunk
        else:
            fulldf=pd.concat([fulldf,chunk])

(467, 78)
(558, 100)
(619, 89)
(879, 151)
(855, 122)
(843, 151)
(747, 138)
(1013, 146)
(831, 135)
(726, 103)
(813, 146)
(816, 159)
(1012, 146)
(753, 157)
(957, 152)
(1025, 154)
(1018, 152)
(920, 140)
(970, 152)
(836, 125)
(883, 141)
(860, 114)
(790, 141)
(974, 164)
(1001, 152)
(1059, 163)
(994, 158)
(1015, 156)
(980, 171)
(1088, 170)
(1063, 181)
(1068, 160)
(996, 166)
(1049, 167)
(1124, 170)
(899, 166)
(624, 106)
(947, 174)
(1147, 173)
(26, 32)


In [325]:
fulldf.shape

(35245, 239)

In [326]:
fulldf = fulldf.groupby(by=['Molecule_ChEMBL_ID','Smiles']).mean()
fulldf.shape

(11480, 239)

In [327]:
# fulldf=fulldf[col_order]
fulldf.to_csv(f'chembl_kinase_{st_type}s_nM.csv')

In [328]:
# fulldf=fulldf.melt(value_name = f'{st_type}_nM', var_name='Kinase')
# fulldf=fulldf.dropna(subset=f'{st_type}_nM')
# col_order = fulldf.Kinase.value_counts().index.tolist()
# fulldf.Kinase.value_counts().to_csv(f'chembl_{st_type}_counts_per_Kinase.csv')

# ChEMBL Lookup gene symbols by target id

In [4]:
import time

In [138]:
ic50=pd.read_csv('ChEMBL_kinase_IC50s_nM.csv')
ic50smi=set(ic50.Smiles)
ic50=ic50.set_index(['Molecule_ChEMBL_ID','Smiles'])
ki=pd.read_csv('ChEMBL_kinase_Kis_nM.csv')
kismi=set(ki.Smiles)
ki=ki.set_index(['Molecule_ChEMBL_ID','Smiles'])
ic50.head(2)

,,CHEMBL279 Vascular endothelial growth factor receptor 2 KDR,CHEMBL203 Epidermal growth factor receptor erbB1 EGFR,CHEMBL2971 Tyrosine-protein kinase JAK2 JAK2,CHEMBL260 MAP kinase p38 alpha MAPK14,CHEMBL5145 Serine/threonine-protein kinase B-raf BRAF,CHEMBL2842 Serine/threonine-protein kinase mTOR MTOR,CHEMBL2835 Tyrosine-protein kinase JAK1 JAK1,CHEMBL2599 Tyrosine-protein kinase SYK SYK,CHEMBL3717 Hepatocyte growth factor receptor MET,CHEMBL2148 Tyrosine-protein kinase JAK3 JAK3,CHEMBL4282 Serine/threonine-protein kinase AKT AKT1,CHEMBL4040 MAP kinase ERK2 MAPK1,CHEMBL267 Tyrosine-protein kinase SRC SRC,CHEMBL2815 Nerve growth factor receptor Trk-A NTRK1,CHEMBL2147 Serine/threonine-protein kinase PIM1 PIM1,CHEMBL5251 Tyrosine-protein kinase BTK BTK,CHEMBL4722 Serine/threonine-protein kinase Aurora-A AURKA,CHEMBL262 Glycogen synthase kinase-3 beta GSK3B,CHEMBL3650 Fibroblast growth factor receptor 1 FGFR1,CHEMBL1957 Insulin-like growth factor I receptor IGF1R,CHEMBL1974 Tyrosine-protein kinase receptor FLT3 FLT3,CHEMBL4523 Serine/threonine-protein kinase PIM2 PIM2,CHEMBL1824 Receptor protein-tyrosine kinase erbB-2 ERBB2,CHEMBL4630 Serine/threonine-protein kinase Chk1 CHEK1,CHEMBL2973 Rho-associated protein kinase 2 ROCK2,CHEMBL3778 Interleukin-1 receptor-associated kinase 4 IRAK4,CHEMBL4247 ALK tyrosine kinase receptor ALK,CHEMBL2185 Serine/threonine-protein kinase Aurora-B AURKB,CHEMBL258 Tyrosine-protein kinase LCK LCK,CHEMBL301 Cyclin-dependent kinase 2 CDK2,CHEMBL4501 Ribosomal protein S6 kinase 1 RPS6KB1,CHEMBL1862 Tyrosine-protein kinase ABL ABL1,CHEMBL2742 Fibroblast growth factor receptor 3 FGFR3,CHEMBL2695 Focal adhesion kinase 1 PTK2,CHEMBL1844 Macrophage colony stimulating factor receptor CSF1R,CHEMBL2276 c-Jun N-terminal kinase 1 MAPK8,CHEMBL1913 Platelet-derived growth factor receptor beta PDGFRB,CHEMBL1936 Stem cell growth factor receptor KIT,CHEMBL1075104 Leucine-rich repeat serine/threonine-protein kinase 2 LRRK2,CHEMBL3553 Tyrosine-protein kinase TYK2 TYK2,CHEMBL1906 Serine/threonine-protein kinase RAF RAF1,CHEMBL2637 c-Jun N-terminal kinase 3 MAPK10,CHEMBL3983 Dual specificity protein kinase TTK TTK,CHEMBL3231 Rho-associated protein kinase 1 ROCK1,CHEMBL3973 Fibroblast growth factor receptor 4 FGFR4,...,CHEMBL3337327 Tau-tubulin kinase 2 TTBK2,CHEMBL4849 Serine/threonine-protein kinase NEK7 NEK7,CHEMBL4175 Tyrosine-protein kinase CTK MATK,CHEMBL5364 Serine/threonine protein kinase NLK NLK,CHEMBL4598 Serine/threonine-protein kinase MST1 STK4,CHEMBL5679 Serine/threonine-protein kinase Nek3 NEK3,CHEMBL3981 Serine/threonine-protein kinase 10 STK10,CHEMBL5552 Serine/threonine-protein kinase 25 STK25,CHEMBL5257 Serine/threonine-protein kinase NEK9 NEK9,CHEMBL2896 cGMP-dependent protein kinase 2 PRKG2,CHEMBL5654 Myosin-IIIB MYO3B,CHEMBL5668 Serine/threonine-protein kinase SRPK2 SRPK2,CHEMBL5912 Serine/threonine-protein kinase 32B STK32B,CHEMBL1075155 Serine/threonine-protein kinase 38 STK38,CHEMBL5334 Non-receptor tyrosine-protein kinase TNK1 TNK1,CHEMBL1163106 Serine/threonine-protein kinase MAK MAK,CHEMBL4627 Myosin light chain kinase MYLK3,CHEMBL5789 Cyclin-dependent kinase-like 1 CDKL1,CHEMBL5388 Serine/threonine-protein kinase tousled-like 1 TLK1,CHEMBL5606 Serine/threonine-protein kinase 11 STK11,CHEMBL5546 Myosin IIIA MYO3A,CHEMBL5579 Citron Rho-interacting kinase CIT,CHEMBL4597 Serine/threonine-protein kinase PCTAIRE-1 CDK16,CHEMBL3627581 Serine/threonine-protein kinase N3 PKN3,CHEMBL5808 PITSLRE serine/threonine-protein kinase CDC2L1 CDK11B,CHEMBL5638 Serine/threonine-protein kinase Nek11 NEK11,CHEMBL1908384 SNF-related serine/threonine-protein kinase SNRK,CHEMBL4853 Mitogen-activated protein kinase kinase kinase 4 MAP3K4,CHEMBL1163126 Serine/threonine-protein kinase ICK CILK1,CHEMBL5970 Mitogen-activated protein kinase kinase kinase 3 MAP3K3,CHEMBL5615 Serine/threonine-protein kinase MRCK gamma CDC42BPG,CHEMBL1938215 Kinase suppressor of Ras 1 KSR1,CHEMBL2417353 Serine/threonine-protein kinase Nek8 NEK8

In [139]:
len(ic50smi.union(kismi))

111221

In [140]:
len(ic50smi), len(kismi)

(101704, 11480)

In [118]:
# sofar=0

In [119]:
target = new_client.target

df=ki
cols=[]
for i, col in enumerate(df.columns[sofar:]):
    targ=col.split('_')[0]
    if i%10==0:
        print(f"On target {i+sofar} / {len(df.columns)}")
    target_info = target.filter(target_chembl_id=targ).only(['target_components'])
    syns = target_info[0]['target_components'][0]['target_component_synonyms']
    gene_syn=[x['component_synonym'] for x in syns if x['syn_type']=='GENE_SYMBOL'][0]
    col=col.replace('_',' ') + ' ' + gene_syn
    cols.append(col)
    time.sleep(1)

On target 0 / 239
On target 10 / 239
On target 20 / 239
On target 30 / 239
On target 40 / 239
On target 50 / 239
On target 60 / 239
On target 70 / 239
On target 80 / 239
On target 90 / 239
On target 100 / 239
On target 110 / 239
On target 120 / 239
On target 130 / 239
On target 140 / 239
On target 150 / 239
On target 160 / 239
On target 170 / 239
On target 180 / 239
On target 190 / 239
On target 200 / 239
On target 210 / 239
On target 220 / 239
On target 230 / 239


In [121]:
len(cols)

239

In [122]:
sofar=sofar+len(cols)
sofar

239

In [123]:
# cols1=cols.copy()
# cols2=cols.copy()
# cols3=cols.copy()
# cols4=cols.copy()
# cols5=cols.copy()
# cols6=cols.copy()
# cols7=cols.copy()

In [124]:
# sum([len(cols1), 
#      len(cols2), 
#      len(cols3), 
#      len(cols4), 
#      len(cols5), 
#      len(cols6), 
#      len(cols7)])

In [125]:
sofar

239

In [105]:
# cols=[]
# cols.extend(cols114)
# cols.extend(cols209)
# cols.extend(cols296)
# cols.extend(cols309)
# cols.extend(cols327)
# cols.extend(cols348)
# cols.extend(cols391)
# len(cols)

391

In [126]:
newcols=[]
for newcol, oldcol in zip(cols, df.columns):
    if oldcol.replace('_',' ') in newcol:
        newcols.append(newcol)
len(newcols)

239

In [127]:
oldcol

'CHEMBL3980_Serine/threonine-protein kinase 17B'

In [128]:
newcol

'CHEMBL3980 Serine/threonine-protein kinase 17B STK17B'

In [129]:
df.columns=cols

In [130]:
st_type='Ki'

In [131]:
df.to_csv(f"ChEMBL_kinase_{st_type}s_nM.csv")

In [136]:
len(set(ki.columns) - set(ic50.columns))

0

In [133]:
ic50

,,CHEMBL279 Vascular endothelial growth factor receptor 2 KDR,CHEMBL203 Epidermal growth factor receptor erbB1 EGFR,CHEMBL2971 Tyrosine-protein kinase JAK2 JAK2,CHEMBL260 MAP kinase p38 alpha MAPK14,CHEMBL5145 Serine/threonine-protein kinase B-raf BRAF,CHEMBL2842 Serine/threonine-protein kinase mTOR MTOR,CHEMBL2835 Tyrosine-protein kinase JAK1 JAK1,CHEMBL2599 Tyrosine-protein kinase SYK SYK,CHEMBL3717 Hepatocyte growth factor receptor MET,CHEMBL2148 Tyrosine-protein kinase JAK3 JAK3,CHEMBL4282 Serine/threonine-protein kinase AKT AKT1,CHEMBL4040 MAP kinase ERK2 MAPK1,CHEMBL267 Tyrosine-protein kinase SRC SRC,CHEMBL2815 Nerve growth factor receptor Trk-A NTRK1,CHEMBL2147 Serine/threonine-protein kinase PIM1 PIM1,CHEMBL5251 Tyrosine-protein kinase BTK BTK,CHEMBL4722 Serine/threonine-protein kinase Aurora-A AURKA,CHEMBL262 Glycogen synthase kinase-3 beta GSK3B,CHEMBL3650 Fibroblast growth factor receptor 1 FGFR1,CHEMBL1957 Insulin-like growth factor I receptor IGF1R,CHEMBL1974 Tyrosine-protein kinase receptor FLT3 FLT3,CHEMBL4523 Serine/threonine-protein kinase PIM2 PIM2,CHEMBL1824 Receptor protein-tyrosine kinase erbB-2 ERBB2,CHEMBL4630 Serine/threonine-protein kinase Chk1 CHEK1,CHEMBL2973 Rho-associated protein kinase 2 ROCK2,CHEMBL3778 Interleukin-1 receptor-associated kinase 4 IRAK4,CHEMBL4247 ALK tyrosine kinase receptor ALK,CHEMBL2185 Serine/threonine-protein kinase Aurora-B AURKB,CHEMBL258 Tyrosine-protein kinase LCK LCK,CHEMBL301 Cyclin-dependent kinase 2 CDK2,CHEMBL4501 Ribosomal protein S6 kinase 1 RPS6KB1,CHEMBL1862 Tyrosine-protein kinase ABL ABL1,CHEMBL2742 Fibroblast growth factor receptor 3 FGFR3,CHEMBL2695 Focal adhesion kinase 1 PTK2,CHEMBL1844 Macrophage colony stimulating factor receptor CSF1R,CHEMBL2276 c-Jun N-terminal kinase 1 MAPK8,CHEMBL1913 Platelet-derived growth factor receptor beta PDGFRB,CHEMBL1936 Stem cell growth factor receptor KIT,CHEMBL1075104 Leucine-rich repeat serine/threonine-protein kinase 2 LRRK2,CHEMBL3553 Tyrosine-protein kinase TYK2 TYK2,CHEMBL1906 Serine/threonine-protein kinase RAF RAF1,CHEMBL2637 c-Jun N-terminal kinase 3 MAPK10,CHEMBL3983 Dual specificity protein kinase TTK TTK,CHEMBL3231 Rho-associated protein kinase 1 ROCK1,CHEMBL3973 Fibroblast growth factor receptor 4 FGFR4,...,CHEMBL3337327 Tau-tubulin kinase 2 TTBK2,CHEMBL4849 Serine/threonine-protein kinase NEK7 NEK7,CHEMBL4175 Tyrosine-protein kinase CTK MATK,CHEMBL5364 Serine/threonine protein kinase NLK NLK,CHEMBL4598 Serine/threonine-protein kinase MST1 STK4,CHEMBL5679 Serine/threonine-protein kinase Nek3 NEK3,CHEMBL3981 Serine/threonine-protein kinase 10 STK10,CHEMBL5552 Serine/threonine-protein kinase 25 STK25,CHEMBL5257 Serine/threonine-protein kinase NEK9 NEK9,CHEMBL2896 cGMP-dependent protein kinase 2 PRKG2,CHEMBL5654 Myosin-IIIB MYO3B,CHEMBL5668 Serine/threonine-protein kinase SRPK2 SRPK2,CHEMBL5912 Serine/threonine-protein kinase 32B STK32B,CHEMBL1075155 Serine/threonine-protein kinase 38 STK38,CHEMBL5334 Non-receptor tyrosine-protein kinase TNK1 TNK1,CHEMBL1163106 Serine/threonine-protein kinase MAK MAK,CHEMBL4627 Myosin light chain kinase MYLK3,CHEMBL5789 Cyclin-dependent kinase-like 1 CDKL1,CHEMBL5388 Serine/threonine-protein kinase tousled-like 1 TLK1,CHEMBL5606 Serine/threonine-protein kinase 11 STK11,CHEMBL5546 Myosin IIIA MYO3A,CHEMBL5579 Citron Rho-interacting kinase CIT,CHEMBL4597 Serine/threonine-protein kinase PCTAIRE-1 CDK16,CHEMBL3627581 Serine/threonine-protein kinase N3 PKN3,CHEMBL5808 PITSLRE serine/threonine-protein kinase CDC2L1 CDK11B,CHEMBL5638 Serine/threonine-protein kinase Nek11 NEK11,CHEMBL1908384 SNF-related serine/threonine-protein kinase SNRK,CHEMBL4853 Mitogen-activated protein kinase kinase kinase 4 MAP3K4,CHEMBL1163126 Serine/threonine-protein kinase ICK CILK1,CHEMBL5970 Mitogen-activated protein kinase kinase kinase 3 MAP3K3,CHEMBL5615 Serine/threonine-protein kinase MRCK gamma CDC42BPG,CHEMBL1938215 Kinase suppressor of Ras 1 KSR1,CHEMBL2417353 Serine/threonine-protein kinase Nek8 NEK8

In [134]:
ki

,,CHEMBL2147 Serine/threonine-protein kinase PIM1 PIM1,CHEMBL2971 Tyrosine-protein kinase JAK2 JAK2,CHEMBL1862 Tyrosine-protein kinase ABL ABL1,CHEMBL262 Glycogen synthase kinase-3 beta GSK3B,CHEMBL1075104 Leucine-rich repeat serine/threonine-protein kinase 2 LRRK2,CHEMBL279 Vascular endothelial growth factor receptor 2 KDR,CHEMBL2835 Tyrosine-protein kinase JAK1 JAK1,CHEMBL4203 Dual specificity protein kinase CLK4 CLK4,CHEMBL2185 Serine/threonine-protein kinase Aurora-B AURKB,CHEMBL6166 Mitogen-activated protein kinase kinase kinase kinase 4 MAP4K4,CHEMBL1908389 Mitogen-activated protein kinase kinase kinase 12 MAP3K12,CHEMBL5888 Mitogen-activated protein kinase kinase kinase 14 MAP3K14,CHEMBL3231 Rho-associated protein kinase 1 ROCK1,CHEMBL4722 Serine/threonine-protein kinase Aurora-A AURKA,CHEMBL5407 Serine/threonine-protein kinase PIM3 PIM3,CHEMBL301 Cyclin-dependent kinase 2 CDK2,CHEMBL3717 Hepatocyte growth factor receptor MET,CHEMBL2973 Rho-associated protein kinase 2 ROCK2,CHEMBL2292 Dual-specificity tyrosine-phosphorylation regulated kinase 1A DYRK1A,CHEMBL4523 Serine/threonine-protein kinase PIM2 PIM2,CHEMBL1974 Tyrosine-protein kinase receptor FLT3 FLT3,CHEMBL2842 Serine/threonine-protein kinase mTOR MTOR,CHEMBL267 Tyrosine-protein kinase SRC SRC,CHEMBL299 Protein kinase C alpha PRKCA,CHEMBL5330 Mitogen-activated protein kinase kinase kinase kinase 2 MAP4K2,CHEMBL4225 Dual specificity protein kinase CLK2 CLK2,CHEMBL2850 Glycogen synthase kinase-3 alpha GSK3A,CHEMBL2595 Protein kinase C nu PRKD3,CHEMBL4101 cAMP-dependent protein kinase alpha-catalytic subunit PRKACA,CHEMBL4630 Serine/threonine-protein kinase Chk1 CHEK1,CHEMBL4036 Cyclin-dependent kinase 5 CDK5,CHEMBL2345 Ribosomal protein S6 kinase alpha 3 RPS6KA3,CHEMBL4482 Serine/threonine-protein kinase PAK 4 PAK4,CHEMBL2148 Tyrosine-protein kinase JAK3 JAK3,CHEMBL258 Tyrosine-protein kinase LCK LCK,CHEMBL3650 Fibroblast growth factor receptor 1 FGFR1,CHEMBL4501 Ribosomal protein S6 kinase 1 RPS6KB1,CHEMBL2996 Protein kinase C delta PRKCD,CHEMBL4852 Mitogen-activated protein kinase kinase kinase kinase 5 MAP4K5,CHEMBL203 Epidermal growth factor receptor erbB1 EGFR,CHEMBL5261 Serine/threonine-protein kinase TAO1 TAOK1,CHEMBL4204 MAP kinase signal-integrating kinase 2 MKNK2,CHEMBL4247 ALK tyrosine kinase receptor ALK,CHEMBL1844 Macrophage colony stimulating factor receptor CSF1R,CHEMBL4708 Serine/threonine-protein kinase MST2 STK3,...,CHEMBL3384 Protein kinase N1 PKN1,CHEMBL3797 Serine-protein kinase ATM ATM,CHEMBL2873 Mitogen-activated protein kinase kinase kinase 10 MAP3K10,CHEMBL3961 MAP kinase p38 beta MAPK11,CHEMBL2073 Tyrosine-protein kinase YES YES1,CHEMBL5776 Mitogen-activated protein kinase kinase kinase 7 MAP3K7,CHEMBL3863 Protein kinase C mu PRKD1,CHEMBL4487 Serine/threonine-protein kinase PAK 2 PAK2,CHEMBL1906 Serine/threonine-protein kinase RAF RAF1,CHEMBL5331 Proto-oncogene tyrosine-protein kinase MER MERTK,"CHEMBL2428 Myosin light chain kinase, smooth muscle MYLK",CHEMBL3861 Pyruvate dehydrogenase kinase isoform 2 PDK2,CHEMBL6005 Serine/threonine-protein kinase 33 STK33,CHEMBL3935 Serine/threonine-protein kinase Aurora-C AURKC,CHEMBL3893 Pyruvate dehydrogenase kinase isoform 3 PDK3,CHEMBL5784 NUAK family SNF1-like kinase 1 NUAK1,CHEMBL5579 Citron Rho-interacting kinase CIT,CHEMBL2343 Serine/threonine-protein kinase Sgk1 SGK1,"CHEMBL3847 AMP-activated protein kinase, beta-1 subunit PRKAB1",CHEMBL4128 Tyrosine-protein kinase TIE-2 TEK,CHEMBL4226 Dual specificity protein kinase CLK3 CLK3,CHEMBL3973 Fibroblast growth factor receptor 4 FGFR4,CHEMBL4224 Dual specificty protein kinase CLK1 CLK1,CHEMBL4142 Fibroblast growth factor receptor 2 FGFR2,CHEMBL5147 Ephrin type-B receptor 4 EPHB4,CHEMBL5285 Mitogen-activated protein kinase kinase kinase 5 MAP3K5,CHEMBL2558 Death-associated protein kinase 1 DAPK1,CHEMBL4311 Serine/threonine-protein kinase PAK6 PAK6,CHEMBL4246 Tyrosine-protein kinase TEC TEC,"CHEMBL5785 Interferon-induced, double-stranded RNA-activated

In [2]:
df=pd.read_csv("ChEMBL_kinase_IC50s_nM.csv")

In [3]:
mtor=df[~df['CHEMBL2842 Serine/threonine-protein kinase mTOR MTOR'].isna()]

In [15]:
mtor_dict=[]
for column in mtor.columns:
    mtor_dict.append((column, len(mtor[~mtor[column].isna()])))

In [18]:
pd.DataFrame(mtor_dict, columns=['kinase', 'overlaps with mtor']).to_csv('mtor_counts.csv')